In [48]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import IPython.display as idis
import seaborn as sns
import copy
import scipy.stats as stat
import statistics
import functools
from pickle import dump
from pickle import load
%matplotlib inline
pd.options.display.max_rows = 999
pd.options.display.max_columns = 300
import functools

from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import xgboost
np.random.seed(1)

In [49]:
start_hospital-----start_icu----end_icu------end_hospital

NameError: name 'start_hospital' is not defined

In [62]:
# lst_range = [(i,i+10) for i in [j for j in range(0,90,10)]]
lst_range =[(0,10),(10,30),(30,60),(60,90)]
lst_range

[(0, 10), (10, 30), (30, 60), (60, 90)]

In [63]:
list_sub_data = []
list_los_data = []
list_sub_model = []
list_test_error = []

In [64]:
working_patient = pd.read_csv('first_vital_patient_data.csv')  
total_los_patient = pd.read_csv('total_los_patients.csv')
# print(total_los_patient)
drop_long_los = total_los_patient.index[total_los_patient['total_los'] > 90].tolist()
working_patient_data = working_patient.drop(drop_long_los)
total_los_patient_data = total_los_patient.drop(drop_long_los)
working_patient_data = working_patient_data.reset_index(drop=True)
total_los_patient_data = total_los_patient_data.reset_index(drop=True)
# print(len(total_los_patient_data))
total_los_np = np.array(total_los_patient_data)
# print(len(total_los))

drop_list = []
for j in range(0,working_patient_data.shape[1]):
    if working_patient_data.iloc[:,j].isnull().sum() / (working_patient_data.shape[0] - len(drop_long_los)) > 0.99:
        drop_list.append(working_patient_data.columns[j])
working_patient_data.drop(drop_list,axis=1,inplace=True)


X_train, X_test, y_train, y_test = train_test_split(working_patient_data, total_los_patient_data, test_size=0.2, random_state=1)

In [65]:
for i in range(0,4):
    X_train_temp = X_train
    y_train_temp = y_train
    drop_long_los = y_train_temp.index[(y_train_temp['total_los'] <= lst_range[i][0]) | (y_train_temp['total_los'] > lst_range[i][1])].tolist()
    temp_working_patient_data = X_train_temp.drop(drop_long_los)
    temp_total_los_data = y_train_temp.drop(drop_long_los)
    temp_working_patient_data = temp_working_patient_data.reset_index(drop=True)
    temp_total_los_data = temp_total_los_data.reset_index(drop=True)
    drop_list = []
    for j in range(0,X_train.shape[1]):
        if temp_working_patient_data.iloc[:,j].isnull().sum() / X_train.shape[0] > 0.99:
            drop_list.append(temp_working_patient_data.columns[j])
    temp_working_patient_data.drop(drop_list,axis=1,inplace=True)
    list_sub_data.append(temp_working_patient_data)
    list_los_data.append(np.array(temp_total_los_data['total_los']))



In [66]:
for i in range(0,4):
    #fill in Nan with 0
    missingvalues = SimpleImputer(missing_values = np.nan, strategy = "constant",fill_value=0)
    missingvalues = missingvalues.fit(list_sub_data[i])
    list_sub_data[i] = missingvalues.transform(list_sub_data[i])
    # scale data with min-max scale
#     mm_scaler = MinMaxScaler()
#     list_sub_data[i] = mm_scaler.fit_transform(list_sub_data[i])
    #training sub model
    
#     model = xgboost.XGBRegressor(colsample_bytree=0.8,
#                  gamma=0.4,                 
#                  learning_rate=0.05,
#                  max_depth=8,
#                  min_child_weight=1.5,
#                  n_estimators=300,                                                                 
#                  reg_alpha=0.8,
#                  subsample=0.4,
#                  early_stopping_rounds=10)
    
    model = SVR(kernel='rbf', verbose=True)
    model.fit(list_sub_data[i], list_los_data[i])
    dump(model, open(str('model_'+str(i)+'.pkl'), 'wb'))

[LibSVM][LibSVM][LibSVM][LibSVM]

In [67]:
missingvalues = SimpleImputer(missing_values = np.nan, strategy = "constant",fill_value=0)
missingvalues = missingvalues.fit(X_test)
X_test = missingvalues.transform(X_test)

y_test = np.array(y_test['total_los'])

In [69]:
for i in range(0,4):
    model = load(open(str('model_'+str(i)+'.pkl'), 'rb'))
    print(model)
    list_sub_model.append(model)

SVR(verbose=True)
SVR(verbose=True)
SVR(verbose=True)
SVR(verbose=True)


In [70]:
for i in range(0,4):
    testing_los = mean_absolute_error(y_test, list_sub_model[i].predict(X_test))
    list_test_error.append(testing_los)

In [71]:
list_test_error

[6.364971056862539, 8.520330397561441, 27.022404517882766, 59.10399751316387]

In [58]:
# list_test_error = []
# for i in range (0,X_test.shape[0]):
#     test_error = []
#     for j in range(0,9):
#         temp_y_test = y_test[i].reshape(-1,1)
#         temp_X_test = X_test[i].reshape(-1,88)
# #         print(temp_X_test.shape)
# #         print(temp_y_test.shape)
#         testing_los = mean_absolute_error(temp_y_test, list_sub_model[j].predict(temp_X_test))
# #         print(testing_los)
#         test_error.append(testing_los)
# #     print(min_test_error_sub_model) 
#     list_test_error.append(statistics.mean(test_error))


In [59]:
print(statistics.mean(list_test_error))

35.850931866526444


In [60]:
for j in range(0,9):
        temp_y_test = y_test[2].reshape(-1,1)
        temp_X_test = X_test[2].reshape(-1,88)
#         print(temp_X_test.shape)
#         print(temp_y_test.shape)
        testing_los = mean_absolute_error(temp_y_test, list_sub_model[j].predict(temp_X_test))
        print(testing_los)
#         if testing_los < min_test_error_sub_model:
#             min_test_error_sub_model = testing_los

59.49259487676768
53.41819642917663
43.26567597569218
34.29955827614243
23.91307492575048
13.497151156634558
3.10627350059346
8.799716670774373
18.62850426130983


In [34]:
y_test[2]

66.8583

In [374]:
len(list_test_error)

6088